## Data cleaning

### Import libraries

In [ ]:
# Import libraries.
import pandas as pd
import numpy as np
import re

## Client sales data

### Load data

In [ ]:
# Load "2021 Client Sales" data.
sales_2021 = pd.read_csv('2021 Client Sales.csv')

# Load "2022 Client Sales" data.
sales_2022 = pd.read_csv('2022 Client Sales.csv')

### View data

In [ ]:
# View sales_2021 head.
sales_2021.head()

In [ ]:
# View sales_2021 tail.
sales_2021.tail()

In [ ]:
# View sales_2022 head.
sales_2022.head()

In [ ]:
# View sales_2022 tail.
sales_2022.tail()

### Data cleaning

#### Validating the data

In [ ]:
# Validate the data function.
def validate_data(df):

    # Check the data types.
    data_types = df.dtypes
    print("Data types:")
    print(data_types)

    # Check the data shape.
    data_shape = df.shape
    print("Data shape:")
    print(data_shape)

    # Check for unique values
    unique_counts = df.nunique()
    print("Unique values per column:")
    print(unique_counts)

    # Check for duplicates
    duplicate_count = df.duplicated().sum()
    print("\nNumber of duplicate rows:")
    print(duplicate_count)

    # Summary statistics
    summary_stats = df.describe()
    print("\nSummary statistics:")
    print(summary_stats)

    return data_types, data_shape, unique_counts, duplicate_count, summary_stats

In [ ]:
# Validating sales_2021.
validate_data(sales_2021)

***Notes on validation***:
- Some incorrect data types. The codes need to be strings. Date needs to be in datetime format.
- Unique values: All provinces & all dates are in the data set (365).
- Max sold units is 245 which seems reasonable. The minimum is a negative number but this could mean refunds - check with stakeholder.
- 175570 rows

In [ ]:
# Validating sales_2022.
validate_data(sales_2022)

***Notes on validation***:
- Some incorrect data types. The codes need to be strings.
- Unique values: All provinces are in the data set. Not all dates are in the data set (348). There are a different number of unique values for the other columns than in the 2021 data (see the below table).
- Max sold units is 717 which seems reasonable especially as this was the year with the heatwaves. The minimum is a negative number but this could mean refunds - check with stakeholder.
- 155415 rows which is less than for the 2021 data (175570 rows)

![Number of unique values.png](attachment:cd9eab62-828a-407d-87fd-2aa6bd776ffa.png)

#### Vertically concatenate sales_2021 and sales_2022

In [ ]:
# Concatenate sales_2021 and sales_2022.
sales = pd.concat([sales_2021, sales_2022.iloc[1:]], ignore_index=True)

In [ ]:
# View sales head.
sales.head()

In [ ]:
# View sales tail.
sales.tail()

In [ ]:
# Check that it's concatenated properly.
sales.to_csv('sales.csv', index=False)

In [ ]:
# Check how many rows and columns in sales.
sales.shape

***Note:*** There are 330984 rows and 8 columns.

#### Clean column names

In [ ]:
# Clean column names function.
def clean_column_names(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    return df

In [ ]:
# Clean column names.
clean_column_names(sales)

# View sales column names.
sales.columns

In [ ]:
# Rename "post_code" to postcode.
sales.rename(columns={'post_code': 'postcode'}, inplace=True)

# View sales column names.
sales.columns

#### Check for missing values (nulls)

In [ ]:
# Create functions for missing values.
def find_nulls(df):
    return df.isnull().sum()

def display_nulls(df, column_name):
    null_rows = df[df[column_name].isnull()]
    display(null_rows)

In [ ]:
# Find nulls.
find_nulls(sales)

***Note:*** There are no missing values (nulls).

#### Changing data types

In [ ]:
# Change the code columns from integers to strings.
# Create function
def convert_to_string(df, columns):
    df[columns] = df[columns].astype(str)
    return df

# Use function.
sales = convert_to_string(sales, ['postcode', 'company_code', 'sales_location'])

# Check data types of sales.
sales.dtypes

In [ ]:
# Convert "date" to datetime, assuming the format day, month, year for all rows.
# If the day, month, or years aren't in this format for all rows, this will be discovering when creating separate date, month, and year columns.

# Create function.
def convert_to_datetime(df, column_name, date_format="%d/%m/%Y"):
    df[column_name] = pd.to_datetime(df[column_name], format=date_format)
    return df

# Use function.
sales = convert_to_datetime(sales, 'date')

# Check data types of sales.
sales.dtypes

In [ ]:
# Function to seperate date, month, and year columns & display unique values
def split_date(df, date_column):
    df['day'] = df[date_column].dt.day
    df['month'] = df[date_column].dt.month
    df['year'] = df[date_column].dt.year

    print("Day:", np.sort(df['day'].unique()))
    print("Month:", np.sort(df['month'].unique()))
    print("Year:", np.sort(df['year'].unique()))

    return df

In [ ]:
# Create separate date, month, and year columns & display unique values
sales = split_date(sales, 'date')

In [ ]:
# Function to create a "month_text" column with the month values as text abbreviations & display unique values

def add_month_text_column(df, date_column):
    # Create the "month_text" column with month abbreviations
    df['month_text'] = df[date_column].dt.month_name().str[:3]

    # View the unique values of month_text
    print(df['month_text'].unique())

    return df

In [ ]:
# Create a "month_text" column with the month values as text abbreviations & display unique values
sales = add_month_text_column(sales, 'date')

#### Create additional columns: day of the week, season

In [ ]:
# Function to create a "day of the week" column & display unique values
def add_day_of_week_column(df, date_column):
    # Create the "day_of_week" column
    df['day_of_week'] = df[date_column].dt.day_name()

    # View the unique values of day_of_week
    print(df['day_of_week'].unique())

    return df

In [ ]:
# Create a "day of the week" column & display unique values
sales = add_day_of_week_column(sales, 'date')

In [ ]:
# Create a function with a mapping dictionary for the seasons & create a new column to display seasons
def add_season_column(df, month_text_column):
    month_season_mapping = {
        'Jan': 'Winter',
        'Feb': 'Winter',
        'Mar': 'Spring',
        'Apr': 'Spring',
        'May': 'Spring',
        'Jun': 'Summer',
        'Jul': 'Summer',
        'Aug': 'Summer',
        'Sep': 'Autumn',
        'Oct': 'Autumn',
        'Nov': 'Autumn',
        'Dec': 'Winter'
    }
    df['season'] = df[month_text_column].replace(month_season_mapping)
    print(df['season'].unique())

    return df

In [ ]:
# Create a season column & check unique values
sales = add_season_column(sales, 'month_text')

#### Cleaning string values

In [ ]:
# Create a function to clean the string values by changing to title text and removing special characters.
def clean_text(df, columns):
        for column in columns:
            df[column] = df[column].str.title()
            df[column] = df[column].str.replace(r'[^a-zA-ZáéíóúñÁÉÍÓÚÑüÜ\s/1-2]', '', regex=True)
        return df

In [ ]:
# View sales again.
sales.head(2)

In [ ]:
# Use the clean_text function of province, town/city, and sales_location.
clean_text(sales, ['province', 'town/city', 'sales_location'])

# View sales.
sales.head()

In [ ]:
# Check unique values in the province column.
sales['province'].unique()

Some of these province names are incorrect.

In [ ]:
# Replace the incorrect province name with the correct one.
# Create a dictionary.
province_dict = {'removed to ensure anonymity of employer'}


# Apply the dictionary to clean province values.
sales['province'] = sales['province'].replace(province_dict)

In [ ]:
# View province unique values (in alphabetical order).
sorted(sales['province'].unique())

In [ ]:
# Check unique values in town/city column.
sales['town/city'].unique()

Some of these towns/cities aren't correct (incorrect spelling or have the province at the end of the name).

In [ ]:
# Replace the incorrect town/city name with the correct one.
# Create a dictionary.
town_cities_dict = {'removed to ensure anonymity of employer'}

# Apply the dictionary to clean town/city values.
sales['town/city'] = sales['town/city'].replace(town_cities_dict)

In [ ]:
# View town/city unique values (in alphabetical order).
sorted(sales['town/city'].unique())

In [ ]:
# View unique sales locations names.
sorted(sales['sales_location'].unique())

In [ ]:
# Remove any extra spaces between words.
# Replace multiple spaces with a single space
sales['sales_location'] = sales['sales_location'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

# View sales_location unique values.
sorted(sales['sales_location'].unique())

In [ ]:
# Change the roman numerals to the correct format e.g. "Ii" to "II".
sales['sales_location'] = (
    sales['sales_location']
    .str.replace('Ii', 'II')
    .str.replace('Iii', 'III')
    .str.replace('IIi', 'III')
    .str.replace('Iv', 'IV'))

# View sales_location unique values.
sorted(sales['sales_location'].unique())

In [ ]:
# Check unique postcodes (should all have X digits).
sales['postcode'].unique()

Not all of these have 5 digits. Are the 4 digit postcodes supposed to have a zero in front?

In [ ]:
# View all wrong amount digit postcodes.
four_digit_postcodes = sales[sales['postcode'].apply(lambda x: isinstance(x, str) and len(x) == Y)]

unique_postcode_province = four_digit_postcodes[['postcode', 'province']].drop_duplicates()

unique_postcode_province

This is a lot of incorrect postcodes so I will get AI to check whether these just require a zero in front. To do this, I will create a dictionary list.

In [ ]:
# Create a list of strings.
postcode_province_list = [f"{postcode}:{province}" for postcode, province in zip(unique_postcode_province['postcode'], unique_postcode_province['province'])]

# View the list.
postcode_province_list

I cross checked this list with the province identifier. I can confirm that a zero needs to be added to all of these postcodes at the start.

In [ ]:
# Add a leading zero to all Y-digit postcodes.

# Create function.
def add_leading_zero(postcode):
    return postcode if len(postcode) == X else '0' + postcode

# Apply the function to the postcode column.
sales['postcode'] = sales['postcode'].apply(add_leading_zero)

# View all unique postcodes.
sales['postcode'].unique()

In [ ]:
# Check that each sales_location only has one unique sales_location code assigned to it.

# Group by station name and check the number of unique station codes
station_code_check = sales.groupby('sales_location')['sales_location_code'].nunique()

# Find sales_location that have more than one unique code
stations_with_multiple_codes = station_code_check[station_code_check > 1]

# View how many sales_location have more than one unique code.
stations_with_multiple_codes

No saleslocations have multiple codes.

#### Check for duplicates.

In [ ]:
# Check for duplicates.
sales.duplicated().sum()

In [ ]:
# View sales.
sales

#### Download sales as a csv.

In [ ]:
# Download as a CSV.
sales.to_csv('sales.csv', index=False)

### Create a filtered sales dataframe for within the 65km radius of metropolitan area

I added leading zeros to the few postcodes that were Y digits long.

I took a sample of postcodes to check whether these were in the 65km radius of the metropolitan area. ***There is a possibility that some postcodes or parts of postcodes are slightly outside of the 65km radius.***

In [ ]:
# Create a list of postcodes to keep.
postcodes_to_keep = ['removed to ensure anonymity of employer']

# Create the filtered_sales dataframe.
filtered_sales = sales[sales['postcode'].isin(postcodes_to_keep)]

# View filtered sales.
filtered_sales

In [ ]:
# Check the provinces that are included. If there are provinces included far from the city, it hasn't been filtered properly.
filtered_sales['province'].unique()

![Screenshot 2025-04-26 154719.png](attachment:f072d902-2da9-4e0a-bd13-e47de69f27aa.png)

Yes, these provinces make sense.

In [ ]:
# Download as a CSV.
filtered_sales.to_csv('filtered_sales.csv', index=False)

## Temperature data

#### Load files

In [ ]:
# Load the CSV files.
temperatures_2021 = pd.read_csv("temperature-city-2021.csv")
temperatures_2022 = pd.read_csv("temperature-city-2022.csv")

#### Vertically concatenating the two data files.

In [ ]:
# Concatenate the two data sets.
temperatures_2021['year'] = 2021
temperatures_2022['year'] = 2022

temperatures = pd.concat([temperatures_2021, temperatures_2022], ignore_index=True)

#### Validating the data.

In [ ]:
# Validate the data.
validate_data(temperatures)

#### Data cleaning

In [ ]:
# Rename columns for easier processing
temperatures.rename(columns={'tavg': 'avg_temp', 'tmin': 'min_temp',
                          'tmax': 'max_temp','prcp': 'precip',
                          'wdir': 'wind_dir','wspd': 'wind_speed',
                          'wpgt': 'wind_peak','pres': 'avg_pressure',
                          'tsun': 'sun_minutes'}, inplace=True)
temperatures.columns

In [ ]:
# Checking for missing values (nulls).
find_nulls(temperatures)

In [ ]:
# Remove columns snow, wind_peak & sun_minutes as they do not contain data (713 out of 713 null values)
temperatures.drop(columns=['snow', 'wind_peak', 'sun_minutes'], inplace=True)

In [ ]:
# Display rows with null values in avg_temp
display_nulls(temperatures, 'avg_temp')

In [ ]:
# Display rows with null values in wind_dir
display_nulls(temperatures, 'wind_dir')

In [ ]:
# Display rows with null values in wind_speed
display_nulls(temperatures, 'wind_speed')

In [ ]:
# Display rows with null values in avg_pressure
display_nulls(temperatures, 'avg_pressure')

***Handling nulls***

Null values will not be removed as the columns still contain valuable data, they will not be changed to 00 or avg values to keep the
data set to avoid misrepresentation.

In [ ]:
# Check if all dates are entered YYYY-MM-DD or if there are inconsistencies.
length_ok = temperatures['date'].astype(str).str.len() == 10
hyphens_ok = temperatures['date'].astype(str).str.count('-') == 2
format_ok = length_ok & hyphens_ok
invalid_dates = temperatures[~format_ok]

if invalid_dates.empty:
    print("All dates are correctly formatted as YYYY-MM-DD.")
else:
    print("The following rows have incorrectly formatted dates:")
    print(invalid_dates)

In [ ]:
# Check if there are any missing dates
temperatures['date'] = pd.to_datetime(temperatures['date'])
full_dates = pd.date_range(start=temperatures['date'].min(), end=temperatures['date'].max())
missing_dates = full_dates.difference(temperatures['date'])

if missing_dates.empty:
    print("There are no missing dates in the dataset.")
else:
    print(f"The following dates are missing: {missing_dates.tolist()}")

In [ ]:
# Convert 'date' to datetime without changing format
temperatures['date'] = pd.to_datetime(temperatures['date'], format='%Y-%m-%d')

In [ ]:
# Create separate date, month, and year columns & display unique values.
temperatures = split_date(temperatures, 'date')

In [ ]:
# Create a "day of the week" column & display unique values
temperatures = add_day_of_week_column(temperatures, 'date')

In [ ]:
# Create a "month_text" column with the month values as text abbreviations & display unique values.
temperatures = add_month_text_column(temperatures, 'date')

In [ ]:
# Create a "season" column & display unique values.
temperatures = add_season_column(temperatures, 'month_text')

In [ ]:
temperatures.head()

In [ ]:
# Reorder the columns for simpler overview
ordered_columns = ['date', 'day', 'day_of_week', 'month', 'month_text', 'season', 'year',
                   'avg_temp', 'min_temp', 'max_temp', 'precip', 'wind_dir', 'wind_speed', 'avg_pressure']

temperatures = temperatures[ordered_columns]

temperatures.head()

In [ ]:
# Function to identify outliers using IQR method and print Min/Max + Outliers
def identify_outliers_iqr(df, column):
    min_value = df[column].min()
    max_value = df[column].max()

    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    out_of_range = (df[column] < lower_bound) | (df[column] > upper_bound)
    outliers = df[out_of_range]

    print(f"Min value for {column}: {min_value}")
    print(f"Max value for {column}: {max_value}")

    if not outliers.empty:
        print("\nOutliers for", column)
        display(outliers)
    else:
        print("\nNo outliers found for", column)

    return outliers

# Get min, max and outliers for the 'min_temp' column.
outliers_min_temp = identify_outliers_iqr(temperatures, 'min_temp')

In [ ]:
# Get min, max and outliers for the 'max_temp' column
outliers_max_temp = identify_outliers_iqr(temperatures, 'max_temp')

In [ ]:
# Get min, max and outliers for the 'avg_temp' column
outliers_avg_temp = identify_outliers_iqr(temperatures, 'avg_temp')

In [ ]:
# Get min, max and outliers for the 'precip' column
outliers_precip = identify_outliers_iqr(temperatures, 'precip')

In [ ]:
# Get min, max and outliers for the 'wind_dir' column
outliers_wind_dir = identify_outliers_iqr(temperatures, 'wind_dir')

In [ ]:
# Get min, max and outliers for the 'wind_speed' column
outliers_wind_speed = identify_outliers_iqr(temperatures, 'wind_speed')

In [ ]:
# Get min, max and outliers for the 'avg_pressure' column
outliers_avg_pressure = identify_outliers_iqr(temperatures, 'avg_pressure')

***Handling Outliers***

The temperature values in the dataset appear realistic for the metropolitan area — plausible given the city's continental climate. Precipitation outliers above x mm/day, while relatively rare, are consistent with occasional events, particularly during autumn storms. Wind speeds peaking at x km/h are on the higher end but not extreme, likely reflecting stormy or exposed conditions. Atmospheric pressure values ranging from x to x hPa are also within normal weather variation for the area. Overall, the outlier values, while relatively infrequent, fall within realistic bounds for the area's climate. Therefore, the outliers will be kept in the dataset, as they represent valid and plausible meteorological extremes.

#### Download temperature CSV.

In [ ]:
# Download as a CSV.
temperatures.to_csv('temperatures.csv', index=False)

## Merge filtered_sales and temperatures.

In [ ]:
# Merge temperatures and filtered_sales.
filtered_sales_temp = filtered_sales.merge(temperatures, on='date', how='left')

# View filtered_sales_temp.
filtered_sales_temp

In [ ]:
# View filtered_sales_temp columns.
filtered_sales_temp.columns

In [ ]:
# Drop all duplicate columns.
filtered_sales_temp = filtered_sales_temp.drop(columns=['day_y', 'day_of_week_y', 'month_y', 'month_text_y', 'season_y', 'year_y'])

# Rename columns.
filtered_sales_temp = filtered_sales_temp.rename(columns={'day_x': 'day','month_x': 'month','month_text_x': 'month_text', 'year_x':'year', 'day_of_week_x':'day_of_week', 'season_x':'season'})

# View filtered_sales_temp columns.
filtered_sales_temp.columns

#### Download filtered_sales_temp.

In [ ]:
# Download as a CSV.
filtered_sales_temp.to_csv('filtered_sales_temp.csv', index=False)